In [12]:
# Importamos las bibliotecas necesarias
import requests
import json
import datetime
import uuid
import copy
import time

print("Inicio del script - " + datetime.datetime.now().strftime("%H:%M:%S"))

# 1. Configuración
base_url = "http://localhost:8080"
username = "admin"
password = "admin"
dag_id = "procesar_facturas_multitask_1"

# Parámetros configurables
num_iteraciones = 1  # Número de veces que se repetirá todo el proceso
facturas_por_trama = 200  # Facturas en cada trama
tramas_por_lote = 1  # Tramas por solicitud a la API
num_tramas = 1  # Número de tramas a generar en cada iteración

print(f"Configuración: {num_iteraciones} iteraciones, {num_tramas} tramas por iteración")
print(f"               {facturas_por_trama} facturas por trama, {tramas_por_lote} tramas por lote")
print(f"Total facturas por iteración: {num_tramas * facturas_por_trama}")

# 2. Obtener token de autenticación
print("\nPASO 1: Obteniendo token de autenticación... - " + datetime.datetime.now().strftime("%H:%M:%S"))
auth_url = f"{base_url}/auth/token"
auth_data = {
    "username": username,
    "password": password
}

start_time = time.time()
response = requests.post(
    auth_url,
    headers={"Content-Type": "application/json"},
    data=json.dumps(auth_data)
)
auth_time = time.time() - start_time

print(f"Tiempo para autenticación: {auth_time:.2f} segundos")

if response.status_code in [200, 201]:
    token_data = response.json()
    token = token_data.get("access_token")
    print("Token obtenido correctamente")
else:
    print(f"Error al obtener token: {response.status_code}")
    print(response.text)
    exit(1)

# 3. Definir la factura base (plantilla para cada factura)
print("\nPASO 2: Definiendo plantillas... - " + datetime.datetime.now().strftime("%H:%M:%S"))
factura_base = {
    "NumeroFactura": "F002-2310255",
    "RucCliente": "PE20100055661",
    "FechaEmision": "2021-02-08T14:51:00-05:00",
    "NumeroGuia": "-",
    "NumeroOC": "-",
    "EstadoCliente": "Bloqueado",
    "FechaRecepcion": "2021-02-08T14:51:00-05:00",
    "FechaCreacion": "2021-02-08T14:51:00-05:00",
    "FechaEnvio": "2021-02-08T14:51:00-05:00",
    "DocumentoERP": "0200000027",
    "TipoFactura": "M",
    "LlaveERP": "15302000000080220211",
    "PeriodoPlazoPago": "30",
    "PlazoPago": "D",
    "FormaPago": "C030",
    "CondicionPago": "C030",
    "IdRegistroTipoEmision": "02",
    "Moneda": "PEN",
    "TipoComprobante": "01",
    "FechaVencimiento": "2021-03-30T14:51:00-05:00",
    "SubTotal": "4000.00",
    "TotalDescuento": "0.000",
    "Impuesto1": "720.00",
    "Impuesto2": "0",
    "Total": "4720.00",
    "Observaciones": "Los dias de Pago son los viernes de cada semana.",
    "FechaPago": "",
    "FechaProgramadaPago": "",
    "ItemFactura": [
        {
            "NumGuiaItem": "E001-53",
            "NumeroItemOC": "00010",
            "NumeroParte": "000000000000605665",
            "DescripcionItem": "SRV COBRANZA",
            "CantidadItem": "1.00",
            "UnidadMedidaItem": "BLD",
            "PrecioUnitario": "2000.00",
            "PrecioTotal": "2000.00",
            "NumeroOcItem": "4590023239",
            "TipoItem": "M"
        },
        {
            "NumGuiaItem": "E001-54",
            "NumeroItemOC": "00020",
            "NumeroParte": "000000000000605666",
            "DescripcionItem": "SRV COBRANZA",
            "CantidadItem": "1.00",
            "UnidadMedidaItem": "BLD",
            "PrecioUnitario": "2000.00",
            "PrecioTotal": "2000.00",
            "NumeroOcItem": "4590023240",
            "TipoItem": "M"
        }
    ],
    "Archivos": [
        {
            "nombre": "leeme2.txt",
            "archivo": "Descripción de archivo",
            "url": "https://storage.microsoft.co/demo2.pdf"
        }
    ]
}

# Estadísticas globales
total_facturas_enviadas = 0
tiempo_total_envio = 0
total_peticiones = 0
tiempo_total_generacion = 0

# Bucle principal para repetir el proceso varias veces
for iteracion in range(num_iteraciones):
    print(f"\n{'='*80}")
    print(f"ITERACIÓN {iteracion + 1} DE {num_iteraciones} - " + datetime.datetime.now().strftime("%H:%M:%S"))
    print(f"{'='*80}")
    
    # 4. Generar múltiples tramas con múltiples facturas cada una
    print(f"\nPASO 3.{iteracion+1}: Generando tramas con múltiples facturas... - " + datetime.datetime.now().strftime("%H:%M:%S"))
    start_time_gen = time.time()
    
    todas_tramas = []
    total_facturas_generadas = 0
    
    for i in range(num_tramas):
        # Crear una nueva trama
        nueva_trama = {
            "FACUPLOADMQ": {
                "RazonSocialProveedor": "RENA WARE DEL PERU S A",
                "RucProveedor": f"PE2010013{1863 + i + iteracion*100}",
                "Factura": []
            }
        }
        
        # Generar las facturas para esta trama
        for j in range(facturas_por_trama):
            # Hacer una copia profunda de la factura base
            nueva_factura = copy.deepcopy(factura_base)
            
            # Modificar la factura para hacerla única
            nueva_factura["NumeroFactura"] = f"F{iteracion+1}{i+1}-{23100 + j}"
            nueva_factura["Observaciones"] = f"Iteración #{iteracion+1}, Trama #{i+1}, factura #{j+1}"
            
            # También vamos a modificar algunos valores en los ítems
            for idx, item in enumerate(nueva_factura["ItemFactura"]):
                item["NumeroOcItem"] = f"459{iteracion+1}{i+1}{j+1}{idx+1}"
                item["PrecioUnitario"] = f"{2000 + (iteracion*1000) + (i*100) + (j*10)}.00"
                item["PrecioTotal"] = f"{2000 + (iteracion*1000) + (i*100) + (j*10)}.00"
            
            # Añadir la factura a la trama
            nueva_trama["FACUPLOADMQ"]["Factura"].append(nueva_factura)
            total_facturas_generadas += 1
        
        # Añadir la trama completa a la lista
        todas_tramas.append(nueva_trama)
    
    gen_time = time.time() - start_time_gen
    tiempo_total_generacion += gen_time
    print(f"Tiempo para generar {num_tramas} tramas con {facturas_por_trama} facturas cada una: {gen_time:.2f} segundos")
    print(f"Total de facturas generadas en esta iteración: {total_facturas_generadas}")
    
    # 5. Enviar las tramas por lotes
    print(f"\nPASO 4.{iteracion+1}: Enviando tramas a Airflow... - " + datetime.datetime.now().strftime("%H:%M:%S"))
    
    total_time_iter = 0
    for i in range(0, num_tramas, tramas_por_lote):
        fin_lote = min(i + tramas_por_lote, num_tramas)
        tramas_lote = todas_tramas[i:fin_lote]
        total_peticiones += 1
        
        # Generar ID y fecha para la ejecución
        current_date = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
        run_id = f"proc_fact_iter{iteracion+1}_lote{i//tramas_por_lote+1}_{uuid.uuid4().hex[:8]}"
        
        # Preparar datos para la petición
        data = {
            "dag_run_id": run_id,
            "logical_date": current_date,
            "conf": {
                "tramas": tramas_lote
            }
        }
        
        # Preparar headers
        headers = {
            "accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f"Bearer {token}"
        }
        
        # Endpoint para ejecutar el DAG
        endpoint = f"{base_url}/api/v2/dags/{dag_id}/dagRuns"
        
        print(f"\nIteración {iteracion+1}, Lote {i//tramas_por_lote + 1} - Enviando {len(tramas_lote)} tramas - " + datetime.datetime.now().strftime("%H:%M:%S"))
        
        # Mostrar información de la petición
        print(f"URL: {endpoint}")
        print(f"ID de ejecución: {run_id}")
        print(f"Número de tramas en este lote: {len(tramas_lote)}")
        print(f"Facturas en la primera trama: {len(tramas_lote[0]['FACUPLOADMQ']['Factura'])}")
        print(f"Primera factura: {tramas_lote[0]['FACUPLOADMQ']['Factura'][0]['NumeroFactura']}")
        
        # Calcular tamaño aproximado de la petición
        request_size = len(json.dumps(data)) / 1024  # en KB
        print(f"Tamaño aproximado de la petición: {request_size:.2f} KB")
        
        # Enviar la petición y medir el tiempo
        start_time = time.time()
        
        try:
            response = requests.post(
                endpoint,
                headers=headers,
                data=json.dumps(data)
            )
            
            request_time = time.time() - start_time
            total_time_iter += request_time
            tiempo_total_envio += request_time
            
            print(f"Tiempo para enviar este lote: {request_time:.2f} segundos")
            
            if response.status_code == 200:
                result = response.json()
                print(f"Respuesta exitosa: {result.get('dag_run_id')}")
                print(f"Estado: {result.get('state')}")
                # Contar las facturas enviadas exitosamente
                facturas_en_lote = sum(len(trama['FACUPLOADMQ']['Factura']) for trama in tramas_lote)
                total_facturas_enviadas += facturas_en_lote
            else:
                print(f"Error al ejecutar el DAG: {response.status_code}")
                print(response.text)
        except Exception as e:
            print(f"Error en la solicitud: {str(e)}")
        
        # Pequeña pausa entre envíos
        time.sleep(1)
    
    # Resumen de la iteración
    print(f"\nRESUMEN DE LA ITERACIÓN {iteracion+1}:")
    print(f"Tiempo de generación: {gen_time:.2f} segundos")
    print(f"Tiempo de envío: {total_time_iter:.2f} segundos")
    print(f"Facturas generadas: {total_facturas_generadas}")
    print(f"Facturas enviadas: {facturas_por_trama * len(tramas_lote) * (num_tramas // tramas_por_lote + (1 if num_tramas % tramas_por_lote > 0 else 0))}")

# Resumen final global
print("\n" + "="*50)
print("RESUMEN GLOBAL:")
print("="*50)
print(f"Tiempo de autenticación: {auth_time:.2f} segundos")
print(f"Tiempo total de generación: {tiempo_total_generacion:.2f} segundos")
print(f"Tiempo total de envío: {tiempo_total_envio:.2f} segundos")
print(f"Tiempo promedio por petición: {tiempo_total_envio/total_peticiones:.2f} segundos")
print(f"Tiempo total del script: {auth_time + tiempo_total_generacion + tiempo_total_envio:.2f} segundos")
print(f"Total facturas enviadas: {total_facturas_enviadas}")
print(f"Total peticiones realizadas: {total_peticiones}")
print("\nFin del script - " + datetime.datetime.now().strftime("%H:%M:%S"))

Inicio del script - 14:40:16
Configuración: 1 iteraciones, 1 tramas por iteración
               200 facturas por trama, 1 tramas por lote
Total facturas por iteración: 200

PASO 1: Obteniendo token de autenticación... - 14:40:16


ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /auth/token (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000184DE86FF00>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))